In [112]:
#Dependencies
import csv
import pandas as pd
import random
import numpy as np
import json

In [230]:
#Convert the players list into a Data Frame
purchase_df = pd.read_json("purchase_data.json")

In [231]:
#Total player count
total_players_df = pd.DataFrame(purchase_df["SN"]).count()
total_players = total_players_df.head().reset_index().rename(columns={'index':'Index',0:'Total Players'})
total_players

,Index,Total Players
0,SN,780


In [115]:
#Count of unique players
total_unique_players = purchase_df["SN"].unique()
unique_df = pd.DataFrame(total_unique_players).count()
unique_players = unique_df.head().reset_index().rename(columns={'index':'Index',0:'Total Unique Players'})
unique_players

,Index,Total Unique Players
0,0,573


In [116]:
#Number of unique items
unique_items = purchase_df["Item ID"].unique()
unique_items_df = pd.DataFrame(pd.DataFrame(unique_items)).count()
unique_items = unique_items_df.head().reset_index().rename(columns={'index':'Index',0:'Number of Unique Items'})
unique_items

,Index,Number of Unique Items
0,0,183


In [117]:
#Calculate Average Purchase Price
average_price_df = pd.DataFrame(purchase_df["Price"]).mean().map("${:,.2f}".format)
average_price = average_price_df.head().reset_index().rename(columns={'index':'Index',0:'Average Price'})
average_price

,Index,Average Price
0,Price,$2.93


In [118]:
#Calculate Total number of purchases
total_num = purchase_df["Price"]
total_num_df = pd.DataFrame(total_num).count()
total_num = total_num_df.head().reset_index().rename(columns={'index':'Index',0:'Total Number of Purchases'})
total_num

,Index,Total Number of Purchases
0,Price,780


In [119]:
#Calculate Total Revenue
rev_total = purchase_df["Price"]
rev_total_df = pd.DataFrame(rev_total).sum().map("${:,.2f}".format)
total_rev = rev_total_df.head().reset_index().rename(columns={'index':'Index',0:'Total Revenue'})
total_rev

,Index,Total Revenue
0,Price,"$2,286.33"


In [120]:
#Average Sale Price and Total Revenue
merge_price_table = pd.merge(average_price, total_rev)
merge_price_table

,Index,Average Price,Total Revenue
0,Price,$2.93,"$2,286.33"


In [121]:
# Filter the DataFrame down only to two columns
gender_and_sn_df= pd.DataFrame(purchase_df[["Gender","SN"]])
# Set the index to be "SN"
gender_and_sn_df = gender_and_sn_df.set_index("SN")

In [122]:
#new Gender Data Frame with unique players
new_df = pd.DataFrame(gender_and_sn_df.groupby('SN').first(check_identical=True))
gender_count = new_df['Gender'].value_counts()
gender_df = pd.DataFrame(gender_count).rename(columns={'Gender':'Total Count'})
gender_df

,Total Count
Male,465
Female,100
Other / Non-Disclosed,8


In [123]:
#Gender percent of total for unique purchases
gender_sum = gender_df["Total Count"].sum()
male_players = gender_df.loc["Male","Total Count"]
female_players = gender_df.loc["Female", "Total Count"]
other_players = gender_df.loc["Other / Non-Disclosed", "Total Count"]
male_percent = ((male_players/gender_sum)*100).round(2)
female_percent = ((female_players/gender_sum)*100).round(2)
other_percent = ((other_players/gender_sum)*100).round(2)
percent = {'Gender': ["Male", "Female", "Other / Non-Diclosed"], 'Percent of Total': [male_percent, female_percent, other_percent]}
percent_df = pd.DataFrame(percent)
percent_df

,Gender,Percent of Total
0,Male,81.15
1,Female,17.45
2,Other / Non-Diclosed,1.40


In [124]:
analysis_gender_count_df = pd.DataFrame(purchase_df.groupby(['Gender']).count())
gender_count_clean_df = pd.DataFrame(analysis_gender_count_df.drop(analysis_gender_count_df.columns[[0, 1, 2, 3]], axis=1).rename(columns={'SN':'Purchase Count'}))
gender_count_clean_df

,Purchase Count
Gender,
Female,136
Male,633
Other / Non-Disclosed,11


In [125]:
analysis_gender_pricesum_df = purchase_df.groupby(['Gender']).sum()
gender_pricesum_clean_df = pd.DataFrame(analysis_gender_pricesum_df.drop(analysis_gender_pricesum_df.columns[[0,1]], axis=1).rename(columns={'Price':'Total Purchase Value'}))
gender_pricesum_clean_df

,Total Purchase Value
Gender,
Female,382.91
Male,1867.68
Other / Non-Disclosed,35.74


In [126]:
#Average Purchase Price Normalized
female_total = gender_pricesum_clean_df.loc["Female","Total Purchase Value"]
male_total =  gender_pricesum_clean_df.loc["Male","Total Purchase Value"]
other_total = gender_pricesum_clean_df.loc["Other / Non-Disclosed","Total Purchase Value"]
female_count = gender_count_clean_df.loc["Female", "Purchase Count"]
male_count = gender_count_clean_df.loc["Male", "Purchase Count"]
other_count = gender_count_clean_df.loc["Other / Non-Disclosed", "Purchase Count"]
average_female_purchase = (female_total/female_count).round(2)
average_male_purchase = (male_total/male_count).round(2)
average_other_purchase = (other_total/other_count).round(2)
female_count_normal = gender_df.loc["Female", "Total Count"]
male_count_normal = gender_df.loc["Male", "Total Count"]
other_count_normal = gender_df.loc["Other / Non-Disclosed", "Total Count"]
average_female_purchase_normal = (female_total/female_count_normal).round(2)
average_male_purchase_normal = (male_total/male_count_normal).round(2)
average_other_purchase_normal = (other_total/other_count_normal).round(2)
average_total = {'Gender': ["Female", "Male", "Other / Non-Diclosed"],\
                 'Average Purchase Price': [average_female_purchase, average_male_purchase, average_other_purchase],\
                'Average Purchase Price Normalized':[average_female_purchase_normal,average_male_purchase_normal,\
                                                     average_other_purchase_normal]}
average_total_df = pd.DataFrame(average_total, columns = \
                                ['Gender', 'Average Purchase Price', 'Average Purchase Price Normalized'])
average_total_df

,Gender,Average Purchase Price,Average Purchase Price Normalized
0,Female,2.82,3.83
1,Male,2.95,4.02
2,Other / Non-Diclosed,3.25,4.47


In [139]:
# Define bins and age groups
bins=[0,9,14,19,24,29,34,39,44,49]
age_groups = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40-44","44+"]
purchase_df["Age Groups"] = pd.cut(purchase_df["Age"], bins, labels = age_groups)

In [143]:
age_groups_df = pd.DataFrame(purchase_df.groupby(['Age Groups']).count())
age_groups_clean_df = pd.DataFrame(age_groups_df.drop(age_groups_df.columns[[0, 1, 2, 3, 4]], axis=1).rename(columns={'SN':'Purchase Count'}))
age_groups_clean_df

,Purchase Count
Age Groups,
<10,28
10-14,35
15-19,133
20-24,336
25-29,125
30-34,64
35-39,42
40-44,16
44+,1


In [188]:
#Normalize age groups
normal_age_group_df = pd.DataFrame(purchase_df.groupby("SN").first(check_identical=True))
normal_age_group = normal_age_group_df['Age Groups'].value_counts()
normal_age_df = pd.DataFrame(normal_age_group).rename(columns={'Age Groups':'Normalized Purchase Count'})
normal_age_df

,Normalized Purchase Count
20-24,259
15-19,100
25-29,87
30-34,47
35-39,27
10-14,23
<10,19
40-44,10
44+,1


In [137]:
age_groups_pricesum = purchase_df.groupby(['Age Groups']).sum()
age_groups_pricesum
age_groups_pricesum_df = pd.DataFrame(age_groups_pricesum.drop(age_groups_pricesum.columns[[0, 1]], axis=1).rename(columns={'Price':'Total Purchase Value'}))
age_groups_pricesum_df

,Total Purchase Value
Age Groups,
<10,83.46
10-14,96.95
15-19,386.42
20-24,978.77
25-29,370.33
30-34,197.25
35-39,119.40
40-44,51.03
44+,2.72


In [198]:
#Average Purchase Price & Normalized
underten_total = age_groups_pricesum_df.loc["<10", "Total Purchase Value"]
underfourteen_total = age_groups_pricesum_df.loc["10-14","Total Purchase Value"]
undernineteen_total = age_groups_pricesum_df.loc["15-19","Total Purchase Value"]
undertwentyfour_total = age_groups_pricesum_df.loc["20-24", "Total Purchase Value"]
undertwentynine_total = age_groups_pricesum_df.loc["25-29", "Total Purchase Value"]
underthirtyfour_total = age_groups_pricesum_df.loc["30-34", "Total Purchase Value"]
underthirtynine_total = age_groups_pricesum_df.loc["35-39", "Total Purchase Value"]
underfortyfour_total = age_groups_pricesum_df.loc["40-44", "Total Purchase Value"]
fortyfourplus_total = age_groups_pricesum_df.loc["44+", "Total Purchase Value"]

underten_count = age_groups_clean_df.loc["<10", "Purchase Count"]
underfourteen_count = age_groups_clean_df.loc["10-14","Purchase Count"]
undernineteen_count = age_groups_clean_df.loc["15-19","Purchase Count"]
undertwentyfour_count = age_groups_clean_df.loc["20-24", "Purchase Count"]
undertwentynine_count = age_groups_clean_df.loc["25-29", "Purchase Count"]
underthirtyfour_count = age_groups_clean_df.loc["30-34", "Purchase Count"]
underthirtynine_count = age_groups_clean_df.loc["35-39", "Purchase Count"]
underfortyfour_count = age_groups_clean_df.loc["40-44", "Purchase Count"]
fortyfourplus_count = age_groups_clean_df.loc["44+", "Purchase Count"]

aveprice_underten = (underten_total/underten_count).round(2)
aveprice_underfourteen = (underfourteen_total/underfourteen_count).round(2)
aveprice_underninteen = (undernineteen_total/undernineteen_count).round(2)
aveprice_undertwentyfour = (undertwentyfour_total/undertwentyfour_count).round(2)
aveprice_undertwentynine = (undertwentynine_total/undertwentynine_count).round(2)
aveprice_underthirtyfour = (underthirtyfour_total/underthirtyfour_count).round(2)
aveprice_underthirtynine = (underthirtynine_total/underthirtynine_count).round(2)
aveprice_underfortyfour = (underfortyfour_total/underfortyfour_count).round(2)
aveprice_fortyfourplus = (fortyfourplus_total/fortyfourplus_count).round(2)

normal_underten_count= normal_age_df.loc["<10", "Normalized Purchase Count"]
normal_underfourteen_count = normal_age_df.loc["10-14", "Normalized Purchase Count"] 
normal_undernineteen_count = normal_age_df.loc["15-19", "Normalized Purchase Count"]
normal_undertwentyfour_count = normal_age_df.loc["20-24", "Normalized Purchase Count"]
normal_undertwentynine_count = normal_age_df.loc["25-29", "Normalized Purchase Count"]
normal_underthirtyfour_count = normal_age_df.loc["30-34", "Normalized Purchase Count"]
normal_underthirtynine_count = normal_age_df.loc["35-39", "Normalized Purchase Count"]
normal_underfortyfour_count = normal_age_df.loc["40-44", "Normalized Purchase Count"]
normal_fortyfourplus_count = normal_age_df.loc["44+", "Normalized Purchase Count"]

normal_aveprice_underten = (underten_total/normal_underten_count).round(2)
normal_aveprice_underfourteen = (underfourteen_total/normal_underfourteen_count).round(2)
normal_aveprice_underninteen = (undernineteen_total/normal_undernineteen_count).round(2)
normal_aveprice_undertwentyfour = (undertwentyfour_total/normal_undertwentyfour_count).round(2)
normal_aveprice_undertwentynine = (undertwentynine_total/normal_undertwentynine_count).round(2)
normal_aveprice_underthirtyfour = (underthirtyfour_total/normal_underthirtyfour_count).round(2)
normal_aveprice_underthirtynine = (underthirtynine_total/normal_underthirtynine_count).round(2)
normal_aveprice_underfortyfour = (underfortyfour_total/normal_underfortyfour_count).round(2)
normal_aveprice_fortyfourplus = (fortyfourplus_total/normal_fortyfourplus_count).round(2)

aveprice_total = {'Age Group': ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44","44+"],\
                  'Average Purchase Price': [aveprice_underten, aveprice_underfourteen, aveprice_underninteen,\
                                             aveprice_undertwentyfour, aveprice_undertwentynine,\
                                             aveprice_underthirtyfour, aveprice_underthirtynine,\
                                             aveprice_underfortyfour, aveprice_fortyfourplus],\
                  'Normalized Average': [normal_aveprice_underten, normal_aveprice_underfourteen,\
                                         normal_aveprice_underninteen, normal_aveprice_undertwentyfour,\
                                         normal_aveprice_undertwentynine, normal_aveprice_underthirtyfour,\
                                         normal_aveprice_underthirtynine, normal_aveprice_underfortyfour,\
                                         normal_aveprice_fortyfourplus]}  

aveprice_total_df = pd.DataFrame(aveprice_total)
aveprice_total_df

,Age Group,Average Purchase Price,Normalized Average
0,<10,2.98,4.39
1,10-14,2.77,4.22
2,15-19,2.91,3.86
3,20-24,2.91,3.78
4,25-29,2.96,4.26
5,30-34,3.08,4.20
6,35-39,2.84,4.42
7,40-44,3.19,5.10
8,44+,2.72,2.72


In [223]:
#Top Spenders
top_spenders_pricesum = purchase_df.groupby(['SN']).sum()
top_spenders_pricesum_df = pd.DataFrame(top_spenders_pricesum.drop(top_spenders_pricesum.columns[[0, 1]], axis=1).rename(columns={'Price':'Total Purchase Value'}))
top_spenders_pricesum_df.sort_values(by=['Total Purchase Value'], ascending=False).head(5)

,Total Purchase Value
SN,
Undirrala66,17.06
Saedue76,13.56
Mindimnya67,12.74
Haellysu29,12.73
Eoda93,11.58


In [224]:
#Most Popular Items
most_popular = purchase_df.groupby(['Item Name']).count()
most_popular_df = pd.DataFrame(most_popular.drop(most_popular.columns[[0, 1, 2, 3, 4]], axis=1).rename(columns={'Age Groups': 'Purchase Count'}))
most_popular_df.sort_values(by=['Purchase Count'], ascending=False).head(5)

,Purchase Count
Item Name,
Final Critic,14
Arcane Gem,11
"Betrayal, Whisper of Grieving Widows",11
Stormcaller,10
Woeful Adamantite Claymore,9


In [229]:
#Most Profitable Items
top_profitable = purchase_df.groupby(['Item Name']).sum()
top_profitable_df = pd.DataFrame(top_profitable.drop(top_profitable.columns[[0, 1]], axis=1).rename(columns={'Price':'Total Purchase Value'}))
top_profitable_df.sort_values(by=['Total Purchase Value'], ascending=False).head(5)

,Total Purchase Value
Item Name,
Final Critic,38.60
Retribution Axe,37.26
Stormcaller,34.65
Spectral Diamond Doomblade,29.75
Orenmir,29.70
